In [ ]:
!pip install simpletransformers
!pip install sentence-transformers

In [1]:
import pickle
data = pickle.load(open('/content/drive/My Drive/hi.pickle','rb'))

In [2]:
import pandas as pd
import numpy as np
import random
import re
import time
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df = pd.DataFrame.from_dict(data)

In [4]:


LE = LabelEncoder()
df['task1'] = LE.fit_transform(df['task_1'])
df['task2'] = LE.fit_transform(df['task_2'])
df.head()



,tweet_id,task_1,task_2,hasoc_id,full_tweet,tweet_raw_text,hashtags,smiley,emoji,url,mentions,numerals,reserved_word,emotext,segmented_hash,task1,task2
0,1127755185944711170,HOF,HATE,hasoc_2020_hi_2250,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो 1...,आदमीं को मारने पर गोडसे आतंकी हो सके है तो सिख...,[],[],[],[],[],"[1, 17000, 5000, 3000]",[],[],[],0,0
1,1123578938406592513,NOT,NONE,hasoc_2020_hi_381,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",": जवाहर यादव , अगर हिम्मत है तो पूरा वीडियो ...",[],[],[],[],"[@Vishesh4, @jawaharyadavbjp]",[],[RT],[],[],1,1
2,1127750010156642304,HOF,HATE,hasoc_2020_hi_1510,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन...,[#भगव],[],[],[],"[@FunKeyBaat, @yadavakhilesh]",[],[RT],[],[भगव],0,0
3,1127660759553597441,HOF,PRFN,hasoc_2020_hi_1010,Yey nina khothani labafazi benu phambili Finis...,Yey nina khothani labafazi benu phambili Finis...,[],[],"[🍑, 🌊]",[],[],[],[],"[peach, water wave]",[],0,3
4,1123487255136489472,HOF,HATE,hasoc_2020_hi_331,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,: जब भी कोई सिकुलर कोंग्रेसी गुलाम चाटुकार चर...,[],[],[],[],[@Rajeshbhanjan2],[],[RT],[],[],0,0


In [5]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['tweet_raw_text'], df['task1'], test_size=0.2, stratify=df['task1'])

In [6]:
train_df  = pd.DataFrame()
train_df['text'] = train_x
train_df['labels'] = train_y

In [7]:
eval_df  = pd.DataFrame()
eval_df['text'] = valid_x
eval_df['labels'] = valid_y

In [74]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
model_args = {
    "num_train_epochs": 5,
    'overwrite_output_dir': True
}
# model = ClassificationModel(
#     "bert", "monsoon-nlp/hindi-tpu-electra", weight=[1.0, 0.25,0.25, 1.0] , num_labels = 4, args=model_args
# )
# model = ClassificationModel(
#     "bert", "sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking",weight=[1,0.4], args=model_args
# )
model = ClassificationModel(
    "bert", "monsoon-nlp/hindi-tpu-electra",weight=[1,0.4], args=model_args
)

Some weights of the model checkpoint at monsoon-nlp/hindi-tpu-electra were not used when initializing BertForSequenceClassification: ['electra.embeddings.word_embeddings.weight', 'electra.embeddings.position_embeddings.weight', 'electra.embeddings.token_type_embeddings.weight', 'electra.embeddings.LayerNorm.weight', 'electra.embeddings.LayerNorm.bias', 'electra.encoder.layer.0.attention.self.query.weight', 'electra.encoder.layer.0.attention.self.query.bias', 'electra.encoder.layer.0.attention.self.key.weight', 'electra.encoder.layer.0.attention.self.key.bias', 'electra.encoder.layer.0.attention.self.value.weight', 'electra.encoder.layer.0.attention.self.value.bias', 'electra.encoder.layer.0.attention.output.dense.weight', 'electra.encoder.layer.0.attention.output.dense.bias', 'electra.encoder.layer.0.attention.output.LayerNorm.weight', 'electra.encoder.layer.0.attention.output.LayerNorm.bias', 'electra.encoder.layer.0.intermediate.dense.weight', 'electra.encoder.layer.0.intermediate.de

In [75]:
 # import shutil
# shutil.rmtree("outputs")
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [76]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)


In [77]:
result

{'eval_loss': 0.8335500177741051,
 'fn': 46,
 'fp': 114,
 'mcc': 0.26441538166652956,
 'tn': 56,
 'tp': 377}

In [78]:
predictions, raw_outputs = model.predict(list(valid_x))

In [79]:
from sklearn.metrics import confusion_matrix 

confusion_matrix(list(valid_y), list(predictions))

array([[ 56, 114],
       [ 46, 377]])

In [80]:
from sklearn.metrics import f1_score
f1_score(list(valid_y), list(predictions), average='macro')

0.6183550006435835

In [50]:
weights = [i/10 + 0.1 for i in range(10)]
resultDict = {}
for i in weights:
  resultDict[i] = {}
for i in weights:
  for j in weights:
    resultDict[i][j] = []

In [51]:
for i in weights:
  for j in weights:
    model = ClassificationModel(
    "bert", "sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking",weight=[i,j], args=model_args
)
    model.train_model(train_df)
    result, model_outputs, wrong_predictions  = model.eval_model(eval_df)
    predictions, raw_outputs = model.predict(list(valid_x))
    f1 = f1_score(list(valid_y), list(predictions), average='macro')
    resultDict[i][j] = [result,f1]


Some weights of the model checkpoint at sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking were not used when initializing BertForSequenceClassification: ['transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.sa_layer_norm.weight', 'transformer.layer.0.sa_layer_norm.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.output_layer_norm.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.layer.1.attention.q_lin.weight', 'transformer.layer.1.attention.q_lin.bias', 'transformer.layer.1.attention.k_lin.weig

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Some weights of the model checkpoint at sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking were not used when initializing BertForSequenceClassification: ['transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.sa_layer_norm.weight', 'transformer.layer.0.sa_layer_norm.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.output_layer_norm.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.layer.1.attention.q_lin.weight', 'transformer.layer.1.attention.q_lin.bias', 'transformer.layer.1.attention.k_lin.weig

...

Some weights of the model checkpoint at sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking were not used when initializing BertForSequenceClassification: ['transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.sa_layer_norm.weight', 'transformer.layer.0.sa_layer_norm.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.output_layer_norm.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.layer.1.attention.q_lin.weight', 'transformer.layer.1.attention.q_lin.bias', 'transformer.layer.1.attention.k_lin.weig

Some weights of the model checkpoint at sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking were not used when initializing BertForSequenceClassification: ['transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.sa_layer_norm.weight', 'transformer.layer.0.sa_layer_norm.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.output_layer_norm.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.layer.1.attention.q_lin.weight', 'transformer.layer.1.attention.q_lin.bias', 'transformer.layer.1.attention.k_lin.weig

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f74ac9a2b38>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f74b5295e10>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    sel

Some weights of the model checkpoint at sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking were not used when initializing BertForSequenceClassification: ['transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.sa_layer_norm.weight', 'transformer.layer.0.sa_layer_norm.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.output_layer_norm.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.layer.1.attention.q_lin.weight', 'transformer.layer.1.attention.q_lin.bias', 'transformer.layer.1.attention.k_lin.weig

KeyboardInterrupt: ignored

In [54]:
pickle.dump(resultDict,open('dump.pkl','wb'))

In [59]:
for i in weights:
  for j in weights:
    if resultDict[i][j]:
      print(i,j,resultDict[i][j][1])

0.1 0.1 0.41633858267716534
0.1 0.2 0.41633858267716534
0.1 0.30000000000000004 0.41633858267716534
0.1 0.4 0.41633858267716534
0.1 0.5 0.41633858267716534
0.1 0.6 0.41633858267716534
0.1 0.7 0.41633858267716534
0.1 0.7999999999999999 0.41633858267716534
0.1 0.9 0.41633858267716534
0.1 1.0 0.41633858267716534
0.2 0.1 0.6013485984354627
0.2 0.2 0.571738083774675
0.2 0.30000000000000004 0.521056882427599
0.2 0.4 0.42259672168928064
0.2 0.5 0.5110286538857968
0.2 0.6 0.4547961998161202
0.2 0.7 0.41633858267716534
0.2 0.7999999999999999 0.41633858267716534
0.2 0.9 0.41633858267716534
0.2 1.0 0.41633858267716534
0.30000000000000004 0.1 0.37892752408881436
0.30000000000000004 0.2 0.6226233229750098
0.30000000000000004 0.30000000000000004 0.41633858267716534
0.30000000000000004 0.4 0.5403166359922577
0.30000000000000004 0.5 0.42259672168928064
0.30000000000000004 0.6 0.5078788063926523
0.30000000000000004 0.7 0.4976295088005473
